<a href="https://colab.research.google.com/github/silverstar0727/study-/blob/master/_posts/practice/simpy_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simpy
시뮬레이션을 위한 패키지이다.



In [1]:
!pip install simpy

In [2]:
# enro
import simpy

def clock(env, name, tick):
  while True:
    print(name, env.now)
    yield env.timeout(tick)

env = simpy.Environment()
env.process(clock(env, 'fast', 0.5))
env.process(clock(env, 'slow',1))

env.run(until = 2)

fast 0
slow 0
fast 0.5
slow 1
fast 1.0
fast 1.5


In [3]:
# process
# Event 인스턴스를 yield하는 제너레이터 함수를 만듦
# 제너레이터 내부에서 activity가 수행될 때 resource에서 request를 보내서  resource를 일정 시간동안 사용한다
# process의 이벤트 시간이 모두 지나면 envent의 value를 받을 수 있다.
import simpy
	    
def car(env):
    """
    자동차 프로세스
    주차하고 여행을 떠남
    parking과 driving 상태를 스위칭함
    """
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
                # 환경에서 timeout 이벤트를 발생시킴(parking_duration동안 휴면)
        yield env.timeout(parking_duration)
	    
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)
	    
env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


In [6]:
# resource
import simpy
            
res = simpy.Resource(env, capacity=1)
            
def print_stats(res):
    print('%d of %d slots are allocated.' % (res.count, res.capacity))
    print('  Users:', res.users)
    print('  Queued events:', res.queue)
            
def user(res):
    print_stats(res)
    with res.request() as req:
        yield req
        print_stats(res)
    print_stats(res)
            
procs = [env.process(user(res)), env.process(user(res))]
env.run(until = 30)

ValueError: ignored

In [7]:
# resource
import simpy

res = simpy.Resource(env, capacity = 1)

def print_stats(res):
  print('%d of %d slots are allocated.' %(res.count, res.capacity))
  print('  Users:', res.users)
  print('  Queued events', res.queue)

def user(res):
  print_stats(res)
  with res.request() as req:
    yield req
    print_stats(res)
  print_stats(res)

procs = [env.process(user(res)), env.process(user(res))]
env.run()

0 of 1 slots are allocated.
  Users: []
  Queued events []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x7efe4b6e0ac8>]
  Queued events []
0 of 1 slots are allocated.
  Users: []
  Queued events []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x7efe4b6e0b70>]
  Queued events []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x7efe4b6e0ac8>]
  Queued events [<Request() object at 0x7efe4b6e0b00>]
0 of 1 slots are allocated.
  Users: []
  Queued events [<Request() object at 0x7efe4b6e0b00>]
1 of 1 slots are allocated.
  Users: [<Request() object at 0x7efe4b6e0b70>]
  Queued events [<Request() object at 0x7efe4b6e0ba8>]
0 of 1 slots are allocated.
  Users: []
  Queued events [<Request() object at 0x7efe4b6e0ba8>]
1 of 1 slots are allocated.
  Users: [<Request() object at 0x7efe4b6e0b00>]
  Queued events []
0 of 1 slots are allocated.
  Users: []
  Queued events []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x7efe4b6e0ba8>]
  Queued e

In [0]:
# 리소스 생성: 용량이 제한된 일종의 컨테이너, 모든 리소스엔 최대용량과 두개의 대기열이 있음(put, get)
import numpy as np

library = simpy.Resource(env, capacity = 2)

def Student(env, id, library, arrive_time):
  yield env.timeout(arrive_time) # 도착시간을 랜덤하게 생성
  with library.request() as req: # 프로세스에 접근시 with 리소스.request() as req 사용
    yield req # resource가 사용 가능할 때까지 기다림
    study_time = np.random.uniform(0, 5)

    yield env.timeout(study_time) # study하는 동안 resource 점유

# 사용시
for i in range(0, 5):
  arrive_time = np.random.uniform(1, 5) # 5명학생 객체 생성
  student = Student(env, i, library, arrive_time) 
  env.process(student) # process에 객체 넘김

env.run() # 30 까지 실행

In [9]:
# priority resource: 프로세스가 각 요청에 대해 우선순위 제공 가능
import simpy

def resource_user(name, env, resource, wait, prio):
  yield env.timeout(wait)
  with resource.request(priority = prio) as req:
    print('%s requesting at %s with priority = %s' %(name, env.now, prio))
    yield req
    print('%s got resource at %s' %(name, env.now))
    yield env.timeout(3)

env = simpy.Environment()
res = simpy.PriorityResource(env, capacity = 1)
p1 = env.process(resource_user(1, env, res, wait = 0, prio = 0))
p2 = env.process(resource_user(2, env, res, wait = 1, prio = 0))
p3 = env.process(resource_user(3, env, res, wait = 2, prio = -1))
env.run()

1 requesting at 0 with priority = 0
1 got resource at 0
2 requesting at 1 with priority = 0
3 requesting at 2 with priority = -1
3 got resource at 3
2 got resource at 6


In [10]:
# preemptive Resource: t새로운 요청이 기존 자원을 내쫓고 점유할 수 있음
import simpy

def resource_user(name, env, resource, wait, prio):
  yield env.timeout(wait)
  with resource.request(priority = prio) as req:
    print('%s requesting at %s with prioity = %s' %(name, env.now, prio))
    yield req
    print('%s got resource at %s' %(name, env.now))

    try:
      yield env.timeout(3)

    except simpy.Interrupt as interrupt:
      by = interrupt.caused.by
      usage = env.now - interrupt.cause.usage_since
      print('%s got preempted by %s at %s after %s' %(name, by, env.now, usage))

env = simpy.Environment()
res = simpy.PreemptiveResource(env, capacity = 1)
p1 = env.process(resource_user(1, env, res, wait = 0, prio = 0))
p2 = env.process(resource_user(2, env, res, wait = 1, prio = 0))
p3 = env.process(resource_user(3, env, res, wait = 2, prio = 0))

env.run()

1 requesting at 0 with prioity = 0
1 got resource at 0
2 requesting at 1 with prioity = 0
3 requesting at 2 with prioity = 0
2 got resource at 3
3 got resource at 6


In [13]:
# Containers: 미분화된 대량 생산, 소비를 모델링, 연속 불연속적 모두 가능, 현재 레벨 검색가능 monitor_tank
import simpy

class GasStation: 
  def __init__(self, env):
    self.fuel_dispensers = simpy.Resource(env, capacity = 2)
    self.gas_tank = simpy.Container(env, init = 100, capacity = 1000)
    self.mon_proc = env.process(self.monitor_tank(env))

  def monitor_tank(self, env): 
    while True:
      if self.gas_tank.level < 100:
        print('Calling tanker at %s' %env.now)
        env.process(tanker(env, self))

      yield env.timeout(15)
  
def tanker(env, gas_station):
  yield env.timeout(10) # Need 10 Minutes to arrive
  print('Tanker arriving at %s' %env.now)
  amout = gas_station.gas_tank.capacity - gas_station.gas_tank.level
  yield gas_station.gas_tank.put(amout)

def car(name, env, gas_station):
  print('Car %s arriving at %s' %(name, env.now))
  with gas_station.fuel_dispensers.request() as req:
    yield req
    print('Car %s starts refueling at %s' %(name, env.now))
    yield gas_station.gas_tank.get(40)
    yield env.timeout(5)
    print('Car %s done refueling at %s' %(name, env.now))

def car_generator(env, gas_station):
  for i in range(4):
    env.process(car(i, env, gas_station))
    yield env.timeout(5)

env = simpy.Environment()
gas_station = GasStation(env)
car_gen = env.process(car_generator(env, gas_station))
env.run(35)

Car 0 arriving at 0
Car 0 starts refueling at 0
Car 1 arriving at 5
Car 0 done refueling at 5
Car 1 starts refueling at 5
Car 2 arriving at 10
Car 1 done refueling at 10
Car 2 starts refueling at 10
Calling tanker at 15
Car 3 arriving at 15
Car 3 starts refueling at 15
Tanker arriving at 25
Car 2 done refueling at 30
Car 3 done refueling at 30


In [16]:
#Stores: object의 생산과 소비를 모델링
import simpy

def producer(env, store):
  for i in range(100):
    yield env.timeout(2)
    yield store.put('spam %s' %i)
    print('Produced spam at', env.now)

def consumer(name, env, sotre):
  while True:
    yield env.timeout(1)
    print(name, 'requesting spam at', env.now)
    item = yield store.get()
    print(name, 'got', item, 'at', env.now)

env = simpy.Environment()
store = simpy.Store(env, capacity = 2)

prod = env.process(producer(env, store))
consumers = [env.process(consumer(i, env, store)) for i in range(2)]

env.run(until = 5)

0 requesting spam at 1
1 requesting spam at 1
Produced spam at 2
0 got spam 0 at 2
0 requesting spam at 3
Produced spam at 4
1 got spam 1 at 4


In [0]:
# filter sotre: 다양한 속성이 있는 기계공장을 모델링
import simpy
from collections import namedtuple

Machine = namedtuple('Machine', 'size, duration')